In [4]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import json
from supabase import create_client
from multiprocessing import Process
import time

load_dotenv()

# Connection to Supabase
url_supabase = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url_supabase, key)

# Get the table
db_data = supabase.table("idx_key_stats").select("").execute()
df_db_data = pd.DataFrame(db_data.data)

cols = df_db_data.columns.tolist()

# Get symbol data
symbol_list = df_db_data['symbol'].tolist()
symbol_list

# Remove the .JK
for i in range (len(symbol_list)):
  symbol_list[i] = symbol_list[i].replace(".JK", "")
       

In [59]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime
import time
from requests_html import AsyncHTMLSession, HTMLSession
import asyncio
import aiohttp
import requests

# Scraping data
BASE_URL = 'https://www.tradingview.com/symbols/IDX-'
TECHNICAL_ENUM = ['sell', 'neutral', 'buy']
ANALYST_ENUM = ['strong_buy', 'buy', 'hold', 'sell', 'strong_sell']

def get_url_page(symbol:str) -> str:
    return f"{BASE_URL}{symbol}"

# async def fetch(session, url):
#     try:
#       async with session.get(url) as response:
#           return await response.text()
#     except:
#       print(f'Fail to fetch from {url}')
#       return None

# async def parse_technical(html) -> dict:
#     soup = BeautifulSoup(html, 'html.parser')
#     # Getting into the data
#     speedometer_containers = soup.findAll("div", {"class": "speedometerWrapper-kg4MJrFB"})
#     summary_technical_data_wrapper = speedometer_containers[1]
#     technical_counters_data_wrapper = summary_technical_data_wrapper.findAll("div", {"class": "counterWrapper-kg4MJrFB"})

#     technical_number_data = []
#     for technical_counter in technical_counters_data_wrapper:
#       # Get the number data
#       technical_counters_number = technical_counter.find("span", {"class": "counterNumber-kg4MJrFB"})
#       print(technical_counters_number)
    
#     return dict()

# async def scrap(symbol: str):
#     url = get_url_page(symbol)
#     technical_url = url+"/technicals/"
#     async with aiohttp.ClientSession() as session:
#         html = await fetch(session, technical_url)
#         if (html is not None):
#           links = await parse_technical(html)
#           # Do something with the results
#           print(links)

# await scrap("GOTO")


def scrap_page(url: str) :
    try:
      session = HTMLSession()
      response = session.get(url)
      response.html.render()

      soup = BeautifulSoup(response.html.html, "html.parser")
      return soup
    except Exception as e:
      print(f"Fail scraping from URL: {url}")
      print(e)
      return None
    

def save_to_json(file_path, data):
  with open(file_path, "w") as output_file:
    json.dump(data, output_file, indent=2)

def scrap_rating_data(symbol: str) -> dict:
    url = get_url_page(symbol)
    result_data = dict()
    result_data['symbol'] = symbol
    technical_rating_dict = None
    analyst_rating_dict = None

    # Scrap technical page
    technical_url = url+"/technicals/"
    soup_tpage = scrap_page(technical_url)
    if (soup_tpage is not None):
      # Getting into the data
      speedometer_containers = soup_tpage.findAll("div", {"class": "speedometerWrapper-kg4MJrFB"})
      summary_technical_data_wrapper = speedometer_containers[1]
      technical_counters_data_wrapper = summary_technical_data_wrapper.findAll("div", {"class": "counterWrapper-kg4MJrFB"})

      technical_number_data = []
      for technical_counter in technical_counters_data_wrapper:
        # Get the number data
        technical_counters_number = technical_counter.find("span", {"class": "counterNumber-kg4MJrFB"})
        print(technical_counters_number)
        
      
      # # Getting technical
      # if (item.text == "More technicals"):
      #   technical_rating_dict = dict()

      #   item.click()
      #   try:
      #     _ = WebDriverWait(driver, 5).until(
      #         EC.presence_of_element_located((By.CLASS_NAME, "speedometerWrapper-kg4MJrFB"))
      #       )
      #     technical_data_wrapper = driver.find_elements(By.CLASS_NAME, "speedometerWrapper-kg4MJrFB")
      #     assert (len(technical_data_wrapper) == 3), "Difference in technical data wrapper detected"

      #     # Summary should be the middle one
      #     summary_technical_data_wrapper = technical_data_wrapper[1]
      #     technical_counters_data_wrapper = summary_technical_data_wrapper.find_element(By.CLASS_NAME, "countersWrapper-kg4MJrFB")
      #     technical_rating_data = technical_counters_data_wrapper.text.split("\n")

      #     # Insert the data to dictionary
      #     start_rating_data_idx = 1
      #     for enum in TECHNICAL_ENUM:
      #       technical_rating_dict[enum] = int(technical_rating_data[start_rating_data_idx])
      #       start_rating_data_idx +=2
        
      #     technical_rating_dict['updated_on'] = (datetime.now()).strftime("%Y-%m-%d %H:%M:%S")

    # # Scrap forecast page (analyst)
    # forecast_url = url+"/forecast/"
    # soup_fpage = scrap_page(forecast_url)
    #       except:
    #         print("Failed to get Technical Data")

    #     # Getting Analyst Rating
    #     if (item.text == "See forecast"):
    #       analyst_rating_dict = dict()
          

    #       item.click()

    #       try:
    #         _ = WebDriverWait(driver, 10).until(
    #             EC.presence_of_element_located((By.CLASS_NAME, "container-zZSa1SHt"))
    #           )
    #         analyst_data_wrapper = driver.find_element(By.CLASS_NAME, "container-zZSa1SHt")

    #         # Get the Value
    #         analyst_data_values = analyst_data_wrapper.find_elements(By.CLASS_NAME,"value-GNeDL9vy")

    #         # Insert the data to dictionary
    #         for idx, enum in enumerate(ANALYST_ENUM):
    #            analyst_rating_dict[enum] = int((analyst_data_values[idx]).text)

    #         analyst_rating_dict['updated_on'] = (datetime.now()).strftime("%Y-%m-%d %H:%M:%S")
    #       except:
    #         print("Failed to get Analyst Data")

    result_data['technical_rating'] = technical_rating_dict
    result_data['analyst_rating'] = analyst_rating_dict
    return result_data

scrap_rating_data("GOTO")

Fail scraping from URL: https://www.tradingview.com/symbols/IDX-GOTO/technicals/
Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.


{'symbol': 'GOTO', 'technical_rating': None, 'analyst_rating': None}

In [38]:
if __name__ == "__main__":
  length_list = len(symbol_list)
  i1 = int(length_list / 4)
  i2 = 2 * i1
  i3 = 3 * i1

  p1 = Process(target=scrap_function, args=(symbol_list[:i1], 1))
  p2 = Process(target=scrap_function, args=(symbol_list[i1:i2], 2))
  p3 = Process(target=scrap_function, args=(symbol_list[i2:i3], 3))
  p4 = Process(target=scrap_function, args=(symbol_list[i3:], 4))

  p1.start()
  p2.start()
  p3.start()
  p4.start()

  p1.join()
  p2.join()
  p3.join()
  p4.join()

  # Merge and upsert to db
  df_merge = combine_data(df_db_data)

  # Convert to json. Remove the index in dataframe
  records = df_merge.to_dict(orient="records")

  # Upsert to db
  try:
    supabase.table("idx_key_stats").upsert(
        records
    ).execute()
    print(
        f"Successfully upserted {len(records)} data to database"
    )
  except Exception as e:
    raise Exception(f"Error upserting to database: {e}")

NameError: name 'scrap_function' is not defined